In [1]:
import numpy as np
import importlib.util
import MetaSet as ms
import importlib.util
from MetaSet import dataCaculate as data

## 全局加载 lumapi 模块
spec = importlib.util.spec_from_file_location("lumapi", "D:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py")
lumapi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(lumapi)

In [2]:
fdtd=lumapi.FDTD(hide=True)

fdtd.save("aaa.fsp")

material2="TiO2 (Titanium Dioxide) - Devore"

ms.setMetaFdtd(fdtd, 0.4e-6, 0.4e-6, 1e-6, -1e-6)
ms.addMetaSource(fdtd, 0.4e-6, 0.4e-6, -0.1e-6, [0.532e-6, 0.800e-6])
ms.classicMonitorGroup(fdtd, 0.4e-6, 0.4e-6, 0.6e-6)

fdtd.run()

In [3]:
def classicDataAcquisition_multyWav(fdtd):
    T=fdtd.transmission("plane")
    T = T.ravel()
    Ex=fdtd.getresult("point", "Ex")
    Ex = Ex.ravel()
    return Ex, T 

Ex,T=classicDataAcquisition_multyWav(fdtd)

print(T)
print(Ex)

[0.99999968 0.99999824 1.00000183 1.00000741 0.9999977 ]
[ 0.63494214-0.77264845j  0.98131891-0.19321711j  0.8790021 +0.47673122j
  0.37053163+0.92890012j -0.31392888+0.95055811j]


In [4]:
T=fdtd.transmission("plane")
T = T.ravel()
Ex=fdtd.getresult("point", "Ex")
Ex = Ex.ravel()

print(T)
print(Ex)

[0.99999968 0.99999824 1.00000183 1.00000741 0.9999977 ]
[ 0.63494214-0.77264845j  0.98131891-0.19321711j  0.8790021 +0.47673122j
  0.37053163+0.92890012j -0.31392888+0.95055811j]


In [5]:
Ex,T=data.classicDataAcquisition_multyWav(fdtd)

print(T)
print(Ex)

<class 'numpy.ndarray'>
[0.99999968 0.99999824 1.00000183 1.00000741 0.9999977 ]
[ 0.63494214-0.77264845j  0.98131891-0.19321711j  0.8790021 +0.47673122j
  0.37053163+0.92890012j -0.31392888+0.95055811j]


In [10]:
import sqlite3
DB_PATH = 'D:/WORK/Achromatic_metalens_design_in_Windows/data/Main.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

query = f"SELECT * FROM Parameter WHERE ID = ?"
cursor.execute(query, (2,))
    
row = cursor.fetchone()  # 获取第一行（因为ID唯一）

print(row)
print(type(row))

parameter=[row[3],row[4],row[5]]
print(parameter)

conn.close()

(2, 1, 0, 7.5e-08, None, None, None, None, None, None, None, None, None, None, None, None)
<class 'tuple'>
[7.5e-08, None, None]
